In [40]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu' #use cuda(gpu) if not available use cpu
print (device)
block_size = 8
batch_size = 4

cpu


In [41]:
with open ('wizard_of_oz.txt', 'r', encoding='utf-8') as f: #use 'wizard_of_oz.txt as a text and encode it with utf-8'
    text = f.read() #let script read the text
chars = sorted(set(text)) #sort all used characters in text
print(chars)
vocab_size = len(chars)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']


In [42]:
string_to_int = { ch:i for i,ch in enumerate(chars) } #make string of characters into intigers (full numbers)
int_to_string = { i:ch for i,ch in enumerate(chars) } #make intigers into string of characters
encode = lambda s: [string_to_int[c] for c in s] #encode the characters into the intigers
decode = lambda l: ''.join([int_to_string[i] for i in l]) #decode the intigers into characters

data = torch.tensor(encode(text), dtype=torch.long) #turns encoded text into tensors
#print(data[:100])

In [43]:
n = int(0.8*len(data)) #take only 80% of the data
train_data = data[:n] #use the 80% for training
val_data = data [n:] #use the remaining 20% for validation

def get_batch(split): #defines a function called get_batch that takes a single argument split, which tells us whether to use the training or validation data
    data = train_data if split == 'train' else val_data #This selects which dataset to use: If split is 'train', use train_data.Otherwise use val_data.
    ix = torch.randint(len(data) - block_size, (batch_size,)) #Each index i represents the start of a chunk of length block_size. We subtract block_size to ensure we don’t go out of bounds.
    print(ix)
    x = torch.stack([data[i:i+block_size] for i in ix]) #For each index i in ix, it slices a chunk from data of length block_size. Then stacks them into a tensor of shape (batch_size, block_size)
    y = torch.stack([data[i+1:i+block_size+1]for i in ix]) #This creates the target batch y, which is the same as x but shifted by one token to the right.The model will try to predict each next token in y given the corresponding token in x
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch('train')
print('inputs:')
#print(x.shape)
print(x)
print('targets:')
print(y)


tensor([ 70556,  39274, 108545,   7065])
inputs:
tensor([[58,  1, 55, 68, 78,  1, 74, 69],
        [78, 68, 74,  1, 54, 71, 58,  1],
        [61, 54, 69, 69, 58, 67, 58, 57],
        [68, 67,  0, 62, 67,  1, 72, 62]])
targets:
tensor([[ 1, 55, 68, 78,  1, 74, 69, 68],
        [68, 74,  1, 54, 71, 58,  1, 54],
        [54, 69, 69, 58, 67, 58, 57,  1],
        [67,  0, 62, 67,  1, 72, 62, 60]])


In [44]:
class BiagramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        # This creates a table that turns each word into a prediction for the next word.
        # It's like saying: "If I see word X, what word should come next?"
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, index, targets=None):
        # Look up each input word in the embedding table.
        # The result is a guess (logits) for the next word at each position.
        logits = self.token_embedding_table(index)  # shape: (B, T, vocab_size)

        if targets is None:
            # If we're just using the model (not training), no need for a loss.
            loss = None
        else:
            # We're training the model, so calculate how wrong the guesses were.
            # First, flatten the logits and targets to make them easier to compare.
            B, T, C = logits.shape 
            logits = logits.view(B*T, C)         # make logits 2D: (all tokens, vocab_size)
            targets = targets.view(B*T)          # flatten targets too: (all tokens)
            # Now calculate how far off the model’s guesses were.
            loss = F.cross_entropy(logits, targets)

        # Return both the guesses and the loss (loss is None during generation)
        return logits, loss

    def generate(self, index, max_new_tokens):
        # This function makes the model keep adding words to the input.
        # Example: start with "The", it might add "cat", then "sat", etc.

        for _ in range(max_new_tokens):
            # Get the model's guess for the next word.
            logits, _ = self.forward(index)  # shape: (B, T, vocab_size)

            # Focus only on the last word's prediction.
            logits = logits[:, -1, :]  # shape: (B, vocab_size)

            # Turn the logits into probabilities (so we can pick randomly but fairly)
            probs = F.softmax(logits, dim=-1)  # shape: (B, vocab_size)

            # Randomly pick the next word using those probabilities.
            index_next = torch.multinomial(probs, num_samples=1)  # shape: (B, 1)

            # Add the new word to the end of the sequence.
            index = torch.cat((index, index_next), dim=1)  # shape: (B, T+1)

        # Return the full sequence, including the new words we added.
        return index


model = BiagramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


ffIJC(GnnDL;
u]﻿Gn)SEj-Pt!XMW*W7[7B)*nLY(Sls,e6.:'﻿v"(eeH?maa
Sl4g33zKUOhTJbN]Eo&?x
rsWrM:nIA7A5[[QL9R]hH_0wl )p _4di[h65&Itae31Zf,kd]n)YXwZ3X6'﻿RXeX1SXXaaeo
z5QhaCp(Yf0*GAs"86V].WO1U,5ycf0Re6KOk'xo7S99hlIriaY﻿K1C'xkif
iG[6JAqC;1"oNCa95hCx(C'ms[zK7Ra)0[gsgArUpvDKH_V]hwI*g
uosdZg5BT8;;]7X6(M:_u3kFA[KPJ_3G6R5&eRU,,Sl2z
Ys﻿s y7Cy:3fQ-aCpifq?gqfhsD8&m4.yO*eR
'OsB_38LK"eMmAPt .;Nv43zG20vW(a]Xu-!!ZYeemeW8HYX:hQGGza3.tq* kfDSXfwbT?*L0J[QSl4Lf38N-]glM:d3BZf2CnuRkw-FM
ix6;HwEbuq7(fQbbcsh?p20,-8:ClJ'VOvi6
